In [1]:
import pandas as pd
import gc
import pickle

In [2]:
train = pickle.load(open("init_train.pkl", "rb"))

In [3]:
# save the train days column
pickle.dump(train["d"].str[2:].astype("int"), open("init_train_d.pkl", "wb"))

In [4]:
# create month_id, an index starting from 1 for the first month and increasing by 1 for every month to come.
train = train.rename(columns = {"id": "id_old"})
train = train.rename(columns = {"product_id": "id"})

dict_year_month_id = {}
id_c = 1 
for year in [2012, 2013, 2014, 2015, 2016]:
    for month in range(1, 13):
        dict_year_month_id[str(year) + "_" + str(month)] = id_c
        id_c = id_c + 1
train["month_id"] = train["year"].astype("str") + "_" +  train["month"].astype("str") 
train["month_id"] = train["month_id"].map(dict_year_month_id)

del dict_year_month_id


In [5]:
train_last_year = train.loc[train.year >=2015].copy()

In [6]:
# generate 'month' column for test files 
test_days = []
for day in range(28):
    test_days.append(pickle.load(open("tests/test_{}.pkl".format(day), "rb")))
    if test_days[day]["month"].iloc[0] == 4:
        test_days[day]["month_id"] = 52
    elif test_days[day]["month"].iloc[0] == 5:
        test_days[day]["month_id"] = 53

In [7]:
# Safety - making sure train_last_year and test_days have the same features - without the target for the test.
for col in train_last_year:
    if col not in test_days[0] and col != "sales":
        train_last_year = train_last_year.drop(col, axis = 1)        
new_cols = list( test_days[0].columns)
new_cols.append("sales")

In [8]:
"""
Adding median and average of previous months as features.
"""
def add_features(df):
    # Median_X_months_ago.
    
    if "sales" not in df.columns: #test
        is_test = True
        df["sales"] = 0
        train_last_year_size = train_last_year.shape[0]
        df_size = df.shape[0]
        df = train_last_year.append(df)
        df_save_id = df["id"].copy()           
        df["id"] = df["id"].str[:-len("_validation")]


    else:
        is_test = False

        
    print("grouping..")
    product_month_sales_median = df.groupby(["id", "month_id"])["sales"].agg("median")
    product_month_sales_mean = df.groupby(["id", "month_id"])["sales"].agg("mean")

    for months_window in range(1,3):    
        print("{} / 2".format(months_window))
        window_month = df["month_id"] - months_window
        monthId_product_list =list(zip(df.id, window_month))
        df["median_{}_months_ago".format(months_window)] = pd.Series(monthId_product_list, 
                                                                     index = df.index).map(product_month_sales_median)
        df["median_{}_months_ago".format(months_window)] = df["median_{}_months_ago".format(months_window)].round(1).astype("float32")

        df["mean_{}_months_ago".format(months_window)] = pd.Series(monthId_product_list, 
                                                                     index = df.index).map(product_month_sales_mean)
        df["mean_{}_months_ago".format(months_window)] = df["mean_{}_months_ago".format(months_window)].round(1).astype("float32")

    del  window_month, monthId_product_list

    month_year_product_mean = df.groupby(["month", "year", "id"])["sales"].agg("mean")

    month_year_product_list =list(zip(df["month"], df["year"] - 1, df.id))
    df["avg_last_year"] = pd.Series(month_year_product_list, index = df.index).map(month_year_product_mean)
    df["avg_last_year"] =  df["avg_last_year"].round(1)

    df["avg_last_year"] = df["avg_last_year"].astype("float32")
    
    if (is_test):
        df["id"] = df_save_id           
        df = df.drop("sales", axis = 1)  
        df = df.tail(df_size)

    print("done")
    
    
    
    return df

In [9]:

for col in train:
    if col not in test_days[0] and col !="sales":
        print(col)
        train = train.drop(col, axis = 1)

new_cols = list( test_days[0].columns)
new_cols.append("sales")

id_old
d


In [10]:
"""
Adding features for the train
"""
train = add_features(train)
pickle.dump(train, open("train.pkl", "wb"))
del train
gc.collect()

grouping..
1 / 2
2 / 2
done


0

In [11]:
# Adding features for the test files.
for day, test_day in enumerate(test_days):
    gc.collect()
    if "sales" in test_day:
        test_day = test_day.drop("sales", axis = 1)  
    test_day = add_features(test_day)
    pickle.dump( test_day, open("tests/test_{}.pkl".format(day), "wb"))
    print("{}/28 tests are ready".format(day + 1))


grouping..
1 / 2
2 / 2
done
1/28 tests are ready
grouping..
1 / 2
2 / 2
done
2/28 tests are ready
grouping..
1 / 2
2 / 2
done
3/28 tests are ready
grouping..
1 / 2
2 / 2
done
4/28 tests are ready
grouping..
1 / 2
2 / 2
done
5/28 tests are ready
grouping..
1 / 2
2 / 2
done
6/28 tests are ready
grouping..
1 / 2
2 / 2
done
7/28 tests are ready
grouping..
1 / 2
2 / 2
done
8/28 tests are ready
grouping..
1 / 2
2 / 2
done
9/28 tests are ready
grouping..
1 / 2
2 / 2
done
10/28 tests are ready
grouping..
1 / 2
2 / 2
done
11/28 tests are ready
grouping..
1 / 2
2 / 2
done
12/28 tests are ready
grouping..
1 / 2
2 / 2
done
13/28 tests are ready
grouping..
1 / 2
2 / 2
done
14/28 tests are ready
grouping..
1 / 2
2 / 2
done
15/28 tests are ready
grouping..
1 / 2
2 / 2
done
16/28 tests are ready
grouping..
1 / 2
2 / 2
done
17/28 tests are ready
grouping..
1 / 2
2 / 2
done
18/28 tests are ready
grouping..
1 / 2
2 / 2
done
19/28 tests are ready
grouping..
1 / 2
2 / 2
done
20/28 tests are ready
grouping.